# Neteja de dades i regressió logística

Emprarem les dades del **Fifa22**. [Enllaç](https://www.kaggle.com/datasets/bryanb/fifa-player-stats-database?resource=download&select=FIFA22_official_data.csv)

> The dataset contains +17k unique players and more than 60 columns, general information and all KPIs the famous videogame offers. As the esport scene keeps rising espacially on FIFA, I thought it can be useful for the community (kagglers and/or gamers)
>
>Context
>
>The data was retrieved thanks to a crawler that I implemented to retrieve:
>
>    Aggregated data such as name of the players, age, country
>    Detailed data such as offensive potential, defense, acceleration
>    I like football a lot and this dataset is for me the opportunity to bring my contribution for the realization of projects that can go from simple analysis to elaboration of strategies on optimal composition under constraints…


L'objectiu d'aquesta **mini** pràctica es prediure la possició del jugador emprant entre totes les característiques disponibles quatre. S'han d'entrenar dos classificadors lineals: **Regressió Logística** i **Perceptró**. 


## Importam llibraries

El que primer fem, com sempre, és importar les llibreries que emprarem. **Pot ser que per fer l'exercici necessitis importar més llibreries**.

In [1]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model

import pandas as pd
import numpy as np

## Llegim les dades

Per llegir les dades emprarem la llibreria de ``pandas``. El fitxer en qüestió és el fitxer que heu descarregat de Kaggle.

In [2]:
df = pd.read_csv("./dades.csv")
df.head()

ID             Name  Age  \
0  212198  Bruno Fernandes   26   
1  209658      L. Goretzka   26   
2  176580        L. Suárez   34   
3  192985     K. De Bruyne   30   
4  224334         M. Acuña   29   

                                              Photo Nationality  \
0  https://cdn.sofifa.com/players/212/198/22_60.png    Portugal   
1  https://cdn.sofifa.com/players/209/658/22_60.png     Germany   
2  https://cdn.sofifa.com/players/176/580/22_60.png     Uruguay   
3  https://cdn.sofifa.com/players/192/985/22_60.png     Belgium   
4  https://cdn.sofifa.com/players/224/334/22_60.png   Argentina   

                                  Flag  Overall  Potential  \
0  https://cdn.sofifa.com/flags/pt.png       88         89   
1  https://cdn.sofifa.com/flags/de.png       87         88   
2  https://cdn.sofifa.com/flags/uy.png       88         88   
3  https://cdn.sofifa.com/flags/be.png       91         91   
4  https://cdn.sofifa.com/flags/ar.png       84         84   

                 Club                                Club Logo  ...  \
0   Manchester United   https://cdn.sofifa.com/teams/11/30.png  ...   
1   FC Bayern München   https://cdn.sofifa.com/teams/21/30.png  ...   
2  Atlético de Madrid  https://cdn.sofifa.com/teams/240/30.png  ...   
3     Manchester City   https://cdn.sofifa.com/teams/10/30.png  ...   
4          Sevilla FC  https://cdn.sofifa.com/teams/481/30.png  ...   

  SlidingTackle GKDiving  GKHandling GKKicking  GKPositioning  GKReflexes  \
0          65.0     12.0        14.0      15.0            8.0        14.0   
1          77.0     13.0         8.0      15.0           11.0         9.0   
2          38.0     27.0        25.0      31.0           33.0        37.0   
3          53.0     15.0        13.0       5.0           10.0        13.0   
4          82.0      8.0        14.0      13.0           13.0        14.0   

   Best Position Best Overall Rating Release Clause DefensiveAwareness  
0            CAM                88.0        €206.9M               72.0  
1             CM                87.0        €160.4M               74.0  
2             ST                88.0         €91.2M               42.0  
3             CM                91.0        €232.2M               68.0  
4             LB                84.0         €77.7M               80.0  

[5 rows x 65 columns]

No podem veure totes les columnes que hi ha. Per fer-ho hem d'emprar una instrucció de pandas.

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
df.head()

ID             Name  Age  \
0  212198  Bruno Fernandes   26   
1  209658      L. Goretzka   26   
2  176580        L. Suárez   34   
3  192985     K. De Bruyne   30   
4  224334         M. Acuña   29   

                                              Photo Nationality  \
0  https://cdn.sofifa.com/players/212/198/22_60.png    Portugal   
1  https://cdn.sofifa.com/players/209/658/22_60.png     Germany   
2  https://cdn.sofifa.com/players/176/580/22_60.png     Uruguay   
3  https://cdn.sofifa.com/players/192/985/22_60.png     Belgium   
4  https://cdn.sofifa.com/players/224/334/22_60.png   Argentina   

                                  Flag  Overall  Potential  \
0  https://cdn.sofifa.com/flags/pt.png       88         89   
1  https://cdn.sofifa.com/flags/de.png       87         88   
2  https://cdn.sofifa.com/flags/uy.png       88         88   
3  https://cdn.sofifa.com/flags/be.png       91         91   
4  https://cdn.sofifa.com/flags/ar.png       84         84   

                 Club                                Club Logo    Value  \
0   Manchester United   https://cdn.sofifa.com/teams/11/30.png  €107.5M   
1   FC Bayern München   https://cdn.sofifa.com/teams/21/30.png     €93M   
2  Atlético de Madrid  https://cdn.sofifa.com/teams/240/30.png   €44.5M   
3     Manchester City   https://cdn.sofifa.com/teams/10/30.png  €125.5M   
4          Sevilla FC  https://cdn.sofifa.com/teams/481/30.png     €37M   

    Wage  Special Preferred Foot  International Reputation  Weak Foot  \
0  €250K     2341          Right                       3.0        3.0   
1  €140K     2314          Right                       4.0        4.0   
2  €135K     2307          Right                       5.0        4.0   
3  €350K     2304          Right                       4.0        5.0   
4   €45K     2292           Left                       2.0        3.0   

   Skill Moves     Work Rate         Body Type Real Face  \
0          4.0    High/ High            Unique       Yes   
1          3.0  High/ Medium            Unique       Yes   
2          3.0  High/ Medium            Unique       Yes   
3          4.0    High/ High            Unique       Yes   
4          4.0    High/ High  Stocky (170-185)        No   

                      Position  Jersey Number        Joined Loaned From  \
0  <span class="pos pos18">CAM           18.0  Jan 30, 2020         NaN   
1  <span class="pos pos11">LDM            8.0   Jul 1, 2018         NaN   
2   <span class="pos pos24">RS            9.0  Sep 25, 2020         NaN   
3  <span class="pos pos13">RCM           17.0  Aug 30, 2015         NaN   
4    <span class="pos pos7">LB           19.0  Sep 14, 2020         NaN   

  Contract Valid Until Height Weight  Crossing  Finishing  HeadingAccuracy  \
0                 2025  179cm   69kg      87.0       83.0             64.0   
1                 2026  189cm   82kg      75.0       82.0             86.0   
2                 2022  182cm   83kg      80.0       93.0             84.0   
3                 2025  181cm   70kg      94.0       82.0             55.0   
4                 2024  172cm   69kg      87.0       66.0             58.0   

   ShortPassing  Volleys  Dribbling  Curve  FKAccuracy  LongPassing  \
0          91.0     87.0       83.0   87.0        87.0         88.0   
1          86.0     69.0       84.0   76.0        75.0         84.0   
2          83.0     90.0       83.0   86.0        82.0         77.0   
3          94.0     82.0       88.0   85.0        83.0         93.0   
4          82.0     68.0       87.0   88.0        75.0         78.0   

   BallControl  Acceleration  SprintSpeed  Agility  Reactions  Balance  \
0         87.0          77.0         73.0     80.0       91.0     79.0   
1         87.0          78.0         83.0     76.0       88.0     71.0   
2         86.0          76.0         69.0     75.0       92.0     78.0   
3         91.0          76.0         76.0     79.0       91.0     78.0   
4         88.0          77.0         76.0     83.0  

In [5]:
df[df["Best Position"] == "GK"]

ID             Name  Age  \
10736   21570  14 Rogério Ceni   40   
11192  210257          Ederson   27   
12350  167495         M. Neuer   35   
12410  163301        J. Orozco   35   
13109  204935      J. Pickford   27   
...       ...              ...  ...   
16704  251433          B. Voll   20   
16705  240558    18 L. Clayton   17   
16706  262846         �. Dobre   20   
16707  241317   21 Xue Qinghao   19   
16708  259646        A. Shaikh   18   

                                                  Photo Nationality  \
10736  https://cdn.sofifa.com/players/021/570/14_60.png      Brazil   
11192  https://cdn.sofifa.com/players/210/257/22_60.png      Brazil   
12350  https://cdn.sofifa.com/players/167/495/22_60.png     Germany   
12410  https://cdn.sofifa.com/players/163/301/22_60.png      Mexico   
13109  https://cdn.sofifa.com/players/204/935/22_60.png     England   
...                                                 ...         ...   
16704  https://cdn.sofifa.com/players/251/433/22_60.png     Germany   
16705  https://cdn.sofifa.com/players/240/558/18_60.png     England   
16706  https://cdn.sofifa.com/players/262/846/22_60.png     Romania   
16707  https://cdn.sofifa.com/players/241/317/21_60.png    China PR   
16708  https://cdn.sofifa.com/players/259/646/22_60.png       India   

                                          Flag  Overall  Potential  \
10736      https://cdn.sofifa.com/flags/br.png       75         75   
11192      https://cdn.sofifa.com/flags/br.png       89         91   
12350      https://cdn.sofifa.com/flags/de.png       90         90   
12410      https://cdn.sofifa.com/flags/mx.png       75         75   
13109  https://cdn.sofifa.com/flags/gb-eng.png       83         85   
...                                        ...      ...        ...   
16704      https://cdn.sofifa.com/flags/de.png       58         69   
16705  https://cdn.sofifa.com/flags/gb-eng.png       53         70   
16706      https://cdn.sofifa.com/flags/ro.png       53         63   
16707      https://cdn.sofifa.com/flags/cn.png       47         60   
16708      https://cdn.sofifa.com/flags/in.png       47         67   

                        Club                                   Club Logo  \
10736              São Paulo     https://cdn.sofifa.com/teams/598/30.png   
11192        Manchester City      https://cdn.sofifa.com/teams/10/30.png   
12350      FC Bayern München      https://cdn.sofifa.com/teams/21/30.png   
12410           Club Tijuana  https://cdn.sofifa.com/teams/111678/30.png   
13109                Everton       https://cdn.sofifa.com/teams/7/30.png   
...                      ...                                         ...   
16704     F.C. Hansa Rostock      https://cdn.sofifa.com/teams/27/30.png   
16705        Cheltenham Town    https://cdn.sofifa.com/teams/1936/30.png   
16706  FC Academica Clinceni  https://cdn.sofifa.com/teams/113391/30.png   
16707    Shanghai Shenhua FC  https://cdn.sofifa.com/teams/110955/30.png   
16708     ATK Mohun Bagan FC  https://cdn.sofifa.com/teams/113146/30.png   

        Value   Wage  Special Preferred Foot  International Reputation  \
10736      €0   €30K     1602          Right                       2.0   
11192    €94M  €200K     1583           Left                       3.0   
12350  €13.5M   €86K     1534          Right                       5.0   
12410   €625K   €10K     1531          Right                       1.0   
13109    €33M   €83K     1495           Left                       3.0   
...       ...    ...      ...            ...                       ...   
16704   €375K   €950      802          Right                       1.0   
16705   €100K    €1K      782          Right                       1.0   
16706   €180K   €550      778          Right                       1.0   
16707   €100K   €700      770          Right                       1.0   
16708   €110K   €500      754          Right                       1.0   

       Weak Foot  Skill Moves       Work Rate 

In [ ]:
df[["Name", "Age"]]

## EDA: *Exploratory data analysis*


## Entrenament

## Avaluació

# Problemes i solucions

Les dades no són netes tenen diferent tipus de problemes i errors. En aquesta part vos deix un exemple de com solucionar cada un d'ells per una columna.

| Problema              	|
|-----------------------	|
| Presència de NaNs     	|
| Dades categòriques    	|
| Renou                 	|
| Molta dimensionalitat 	|


## Presència de NaNs

La presència de NaNs nos ens permet emprar les dades per fer prediccions. Disposam de tres estratègies per resoldre aquest problema:

- **Eliminar les files amb NaNs**
- **Substituir per 0**
- **Substituir per una mesura d'agregació**

In [6]:
df.dropna(subset = ['Position'], inplace=True) # Eliminar les files amb NaNs
df['Loaned From'].fillna(value=0, inplace=True)

release_claus_m = df['Composure'].mean()

df['Composure'].fillna(value=release_claus_m, inplace=True)

## Dades categòriques

Tenim dues estratègies possibles:

- **Substituir els valors per enters**
- **One hot encoding**


![OHE](https://www.statology.org/wp-content/uploads/2021/09/oneHot1.png)

In [7]:
clb = df.pop("Club")
ohe_clb = pd.get_dummies(clb, prefix='clb')
df = pd.concat([df.reset_index(drop=True), ohe_clb.reset_index(drop=True)], axis=1, sort=False)

df.head()

ID             Name  Age  \
0  212198  Bruno Fernandes   26   
1  209658      L. Goretzka   26   
2  176580        L. Suárez   34   
3  192985     K. De Bruyne   30   
4  224334         M. Acuña   29   

                                              Photo Nationality  \
0  https://cdn.sofifa.com/players/212/198/22_60.png    Portugal   
1  https://cdn.sofifa.com/players/209/658/22_60.png     Germany   
2  https://cdn.sofifa.com/players/176/580/22_60.png     Uruguay   
3  https://cdn.sofifa.com/players/192/985/22_60.png     Belgium   
4  https://cdn.sofifa.com/players/224/334/22_60.png   Argentina   

                                  Flag  Overall  Potential  \
0  https://cdn.sofifa.com/flags/pt.png       88         89   
1  https://cdn.sofifa.com/flags/de.png       87         88   
2  https://cdn.sofifa.com/flags/uy.png       88         88   
3  https://cdn.sofifa.com/flags/be.png       91         91   
4  https://cdn.sofifa.com/flags/ar.png       84         84   

                                 Club Logo    Value   Wage  Special  \
0   https://cdn.sofifa.com/teams/11/30.png  €107.5M  €250K     2341   
1   https://cdn.sofifa.com/teams/21/30.png     €93M  €140K     2314   
2  https://cdn.sofifa.com/teams/240/30.png   €44.5M  €135K     2307   
3   https://cdn.sofifa.com/teams/10/30.png  €125.5M  €350K     2304   
4  https://cdn.sofifa.com/teams/481/30.png     €37M   €45K     2292   

  Preferred Foot  International Reputation  Weak Foot  Skill Moves  \
0          Right                       3.0        3.0          4.0   
1          Right                       4.0        4.0          3.0   
2          Right                       5.0        4.0          3.0   
3          Right                       4.0        5.0          4.0   
4           Left                       2.0        3.0          4.0   

      Work Rate         Body Type Real Face                     Position  \
0    High/ High            Unique       Yes  <span class="pos pos18">CAM   
1  High/ Medium            Unique       Yes  <span class="pos pos11">LDM   
2  High/ Medium            Unique       Yes   <span class="pos pos24">RS   
3    High/ High            Unique       Yes  <span class="pos pos13">RCM   
4    High/ High  Stocky (170-185)        No    <span class="pos pos7">LB   

   Jersey Number        Joined Loaned From Contract Valid Until Height Weight  \
0           18.0  Jan 30, 2020           0                 2025  179cm   69kg   
1            8.0   Jul 1, 2018           0                 2026  189cm   82kg   
2            9.0  Sep 25, 2020           0                 2022  182cm   83kg   
3           17.0  Aug 30, 2015           0                 2025  181cm   70kg   
4           19.0  Sep 14, 2020           0                 2024  172cm   69kg   

   Crossing  Finishing  HeadingAccuracy  ShortPassing  Volleys  Dribbling  \
0      87.0       83.0             64.0          91.0     87.0       83.0   
1      75.0       82.0             86.0          86.0     69.0       84.0   
2      80.0       93.0             84.0          83.0     90.0       83.0   
3      94.0       82.0             55.0          94.0     82.0       88.0   
4      87.0       66.0             58.0          82.0     68.0       87.0   

   Curve  FKAccuracy  LongPassing  BallControl  Acceleration  SprintSpeed  \
0   87.0        87.0         88.0         87.0          77.0         73.0   
1   76.0        75.0         84.0         87.0          78.0         83.0   
2   86.0        82.0         77.0         86.0          76.0         69.0   
3   85.0        83.0         93.0         91.0          76.0         76.0   
4   88.0        75.0         78.0         88.0          77.0         76.0   

   Agility  Reactions  Balance  ShotPower  Jumping  Stamina  Strength  \
0     80.0       91.0     79.0       89.0     73.0     91.0      70.0   
1     76.0       88.0     71.0       85.0     79.0     88.0      88.0   
2     75.0       92.0     78.0       89.0     69.0     78.0      85.0   
3     79.0

In [8]:
relacio_pos = {}

for value, key in enumerate(df["Best Position"].unique()):
    relacio_pos[key] = value

print(relacio_pos)

df["Best Position"] = df["Best Position"].apply(lambda x: relacio_pos[x])

{'CAM': 0, 'CM': 1, 'ST': 2, 'LB': 3, 'CDM': 4, 'CB': 5, 'RB': 6, 'LM': 7, 'RW': 8, 'LW': 9, 'CF': 10, 'LWB': 11, 'RM': 12, 'RWB': 13, 'GK': 14}


In [9]:
df.head()

ID             Name  Age  \
0  212198  Bruno Fernandes   26   
1  209658      L. Goretzka   26   
2  176580        L. Suárez   34   
3  192985     K. De Bruyne   30   
4  224334         M. Acuña   29   

                                              Photo Nationality  \
0  https://cdn.sofifa.com/players/212/198/22_60.png    Portugal   
1  https://cdn.sofifa.com/players/209/658/22_60.png     Germany   
2  https://cdn.sofifa.com/players/176/580/22_60.png     Uruguay   
3  https://cdn.sofifa.com/players/192/985/22_60.png     Belgium   
4  https://cdn.sofifa.com/players/224/334/22_60.png   Argentina   

                                  Flag  Overall  Potential  \
0  https://cdn.sofifa.com/flags/pt.png       88         89   
1  https://cdn.sofifa.com/flags/de.png       87         88   
2  https://cdn.sofifa.com/flags/uy.png       88         88   
3  https://cdn.sofifa.com/flags/be.png       91         91   
4  https://cdn.sofifa.com/flags/ar.png       84         84   

                                 Club Logo    Value   Wage  Special  \
0   https://cdn.sofifa.com/teams/11/30.png  €107.5M  €250K     2341   
1   https://cdn.sofifa.com/teams/21/30.png     €93M  €140K     2314   
2  https://cdn.sofifa.com/teams/240/30.png   €44.5M  €135K     2307   
3   https://cdn.sofifa.com/teams/10/30.png  €125.5M  €350K     2304   
4  https://cdn.sofifa.com/teams/481/30.png     €37M   €45K     2292   

  Preferred Foot  International Reputation  Weak Foot  Skill Moves  \
0          Right                       3.0        3.0          4.0   
1          Right                       4.0        4.0          3.0   
2          Right                       5.0        4.0          3.0   
3          Right                       4.0        5.0          4.0   
4           Left                       2.0        3.0          4.0   

      Work Rate         Body Type Real Face                     Position  \
0    High/ High            Unique       Yes  <span class="pos pos18">CAM   
1  High/ Medium            Unique       Yes  <span class="pos pos11">LDM   
2  High/ Medium            Unique       Yes   <span class="pos pos24">RS   
3    High/ High            Unique       Yes  <span class="pos pos13">RCM   
4    High/ High  Stocky (170-185)        No    <span class="pos pos7">LB   

   Jersey Number        Joined Loaned From Contract Valid Until Height Weight  \
0           18.0  Jan 30, 2020           0                 2025  179cm   69kg   
1            8.0   Jul 1, 2018           0                 2026  189cm   82kg   
2            9.0  Sep 25, 2020           0                 2022  182cm   83kg   
3           17.0  Aug 30, 2015           0                 2025  181cm   70kg   
4           19.0  Sep 14, 2020           0                 2024  172cm   69kg   

   Crossing  Finishing  HeadingAccuracy  ShortPassing  Volleys  Dribbling  \
0      87.0       83.0             64.0          91.0     87.0       83.0   
1      75.0       82.0             86.0          86.0     69.0       84.0   
2      80.0       93.0             84.0          83.0     90.0       83.0   
3      94.0       82.0             55.0          94.0     82.0       88.0   
4      87.0       66.0             58.0          82.0     68.0       87.0   

   Curve  FKAccuracy  LongPassing  BallControl  Acceleration  SprintSpeed  \
0   87.0        87.0         88.0         87.0          77.0         73.0   
1   76.0        75.0         84.0         87.0          78.0         83.0   
2   86.0        82.0         77.0         86.0          76.0         69.0   
3   85.0        83.0         93.0         91.0          76.0         76.0   
4   88.0        75.0         78.0         88.0          77.0         76.0   

   Agility  Reactions  Balance  ShotPower  Jumping  Stamina  Strength  \
0     80.0       91.0     79.0       89.0     73.0     91.0      70.0   
1     76.0       88.0     71.0       85.0     79.0     88.0      88.0   
2     75.0       92.0     78.0       89.0     69.0     78.0      85.0   
3     79.0

## Renou

Hi ha dades que contenen renou i altres que ho són directament. Les columnes de ID i Name, per exemple no serveixen de res, per tant les podem eliminar. 

In [10]:
df.drop(['ID', 'Name'], inplace=True, axis=1)
df.head()

Age                                             Photo Nationality  \
0   26  https://cdn.sofifa.com/players/212/198/22_60.png    Portugal   
1   26  https://cdn.sofifa.com/players/209/658/22_60.png     Germany   
2   34  https://cdn.sofifa.com/players/176/580/22_60.png     Uruguay   
3   30  https://cdn.sofifa.com/players/192/985/22_60.png     Belgium   
4   29  https://cdn.sofifa.com/players/224/334/22_60.png   Argentina   

                                  Flag  Overall  Potential  \
0  https://cdn.sofifa.com/flags/pt.png       88         89   
1  https://cdn.sofifa.com/flags/de.png       87         88   
2  https://cdn.sofifa.com/flags/uy.png       88         88   
3  https://cdn.sofifa.com/flags/be.png       91         91   
4  https://cdn.sofifa.com/flags/ar.png       84         84   

                                 Club Logo    Value   Wage  Special  \
0   https://cdn.sofifa.com/teams/11/30.png  €107.5M  €250K     2341   
1   https://cdn.sofifa.com/teams/21/30.png     €93M  €140K     2314   
2  https://cdn.sofifa.com/teams/240/30.png   €44.5M  €135K     2307   
3   https://cdn.sofifa.com/teams/10/30.png  €125.5M  €350K     2304   
4  https://cdn.sofifa.com/teams/481/30.png     €37M   €45K     2292   

  Preferred Foot  International Reputation  Weak Foot  Skill Moves  \
0          Right                       3.0        3.0          4.0   
1          Right                       4.0        4.0          3.0   
2          Right                       5.0        4.0          3.0   
3          Right                       4.0        5.0          4.0   
4           Left                       2.0        3.0          4.0   

      Work Rate         Body Type Real Face                     Position  \
0    High/ High            Unique       Yes  <span class="pos pos18">CAM   
1  High/ Medium            Unique       Yes  <span class="pos pos11">LDM   
2  High/ Medium            Unique       Yes   <span class="pos pos24">RS   
3    High/ High            Unique       Yes  <span class="pos pos13">RCM   
4    High/ High  Stocky (170-185)        No    <span class="pos pos7">LB   

   Jersey Number        Joined Loaned From Contract Valid Until Height Weight  \
0           18.0  Jan 30, 2020           0                 2025  179cm   69kg   
1            8.0   Jul 1, 2018           0                 2026  189cm   82kg   
2            9.0  Sep 25, 2020           0                 2022  182cm   83kg   
3           17.0  Aug 30, 2015           0                 2025  181cm   70kg   
4           19.0  Sep 14, 2020           0                 2024  172cm   69kg   

   Crossing  Finishing  HeadingAccuracy  ShortPassing  Volleys  Dribbling  \
0      87.0       83.0             64.0          91.0     87.0       83.0   
1      75.0       82.0             86.0          86.0     69.0       84.0   
2      80.0       93.0             84.0          83.0     90.0       83.0   
3      94.0       82.0             55.0          94.0     82.0       88.0   
4      87.0       66.0             58.0          82.0     68.0       87.0   

   Curve  FKAccuracy  LongPassing  BallControl  Acceleration  SprintSpeed  \
0   87.0        87.0         88.0         87.0          77.0         73.0   
1   76.0        75.0         84.0         87.0          78.0         83.0   
2   86.0        82.0         77.0         86.0          76.0         69.0   
3   85.0        83.0         93.0         91.0          76.0         76.0   
4   88.0        75.0         78.0         88.0          77.0         76.0   

   Agility  Reactions  Balance  ShotPower  Jumping  Stamina  Strength  \
0     80.0       91.0     79.0       89.0     73.0     91.0      70.0   
1     76.0       88.0     71.0       85.0     79.0     88.0      88.0   
2     75.0       92.0     78.0       89.0     69.0     78.0      85.0   
3     79.0       91.0     78.0       91.0     63.0     89.0      74.0   
4     83.0       83.0     90.0       82.0     63.0     90.0      80.0   

   LongShots  Aggression  Interceptions 

Després tenim informació important que conté renou com el Value. Per solucionar-ho podem aplicar-hi una funció ``python`` que corregeixi els problemes que té.

In [11]:
def value_to_float(x):
    """
    From K and M to float.
    
    """
    x = x.replace('€', '')
    ret_val = 0.0
    
    if type(x) == float or type(x) == int:
        ret_val = x
    if 'K' in x:
        if len(x) > 1:
            ret_val = float(x.replace('K', ''))
        ret_val = ret_val *1000
    if 'M' in x:
        if len(x) > 1:
            ret_val = float(x.replace('M', ''))
        ret_val = ret_val * 1000000.0
    return ret_val

df["Value"] = df["Value"].apply(value_to_float)
df.head()

Age                                             Photo Nationality  \
0   26  https://cdn.sofifa.com/players/212/198/22_60.png    Portugal   
1   26  https://cdn.sofifa.com/players/209/658/22_60.png     Germany   
2   34  https://cdn.sofifa.com/players/176/580/22_60.png     Uruguay   
3   30  https://cdn.sofifa.com/players/192/985/22_60.png     Belgium   
4   29  https://cdn.sofifa.com/players/224/334/22_60.png   Argentina   

                                  Flag  Overall  Potential  \
0  https://cdn.sofifa.com/flags/pt.png       88         89   
1  https://cdn.sofifa.com/flags/de.png       87         88   
2  https://cdn.sofifa.com/flags/uy.png       88         88   
3  https://cdn.sofifa.com/flags/be.png       91         91   
4  https://cdn.sofifa.com/flags/ar.png       84         84   

                                 Club Logo        Value   Wage  Special  \
0   https://cdn.sofifa.com/teams/11/30.png  107500000.0  €250K     2341   
1   https://cdn.sofifa.com/teams/21/30.png   93000000.0  €140K     2314   
2  https://cdn.sofifa.com/teams/240/30.png   44500000.0  €135K     2307   
3   https://cdn.sofifa.com/teams/10/30.png  125500000.0  €350K     2304   
4  https://cdn.sofifa.com/teams/481/30.png   37000000.0   €45K     2292   

  Preferred Foot  International Reputation  Weak Foot  Skill Moves  \
0          Right                       3.0        3.0          4.0   
1          Right                       4.0        4.0          3.0   
2          Right                       5.0        4.0          3.0   
3          Right                       4.0        5.0          4.0   
4           Left                       2.0        3.0          4.0   

      Work Rate         Body Type Real Face                     Position  \
0    High/ High            Unique       Yes  <span class="pos pos18">CAM   
1  High/ Medium            Unique       Yes  <span class="pos pos11">LDM   
2  High/ Medium            Unique       Yes   <span class="pos pos24">RS   
3    High/ High            Unique       Yes  <span class="pos pos13">RCM   
4    High/ High  Stocky (170-185)        No    <span class="pos pos7">LB   

   Jersey Number        Joined Loaned From Contract Valid Until Height Weight  \
0           18.0  Jan 30, 2020           0                 2025  179cm   69kg   
1            8.0   Jul 1, 2018           0                 2026  189cm   82kg   
2            9.0  Sep 25, 2020           0                 2022  182cm   83kg   
3           17.0  Aug 30, 2015           0                 2025  181cm   70kg   
4           19.0  Sep 14, 2020           0                 2024  172cm   69kg   

   Crossing  Finishing  HeadingAccuracy  ShortPassing  Volleys  Dribbling  \
0      87.0       83.0             64.0          91.0     87.0       83.0   
1      75.0       82.0             86.0          86.0     69.0       84.0   
2      80.0       93.0             84.0          83.0     90.0       83.0   
3      94.0       82.0             55.0          94.0     82.0       88.0   
4      87.0       66.0             58.0          82.0     68.0       87.0   

   Curve  FKAccuracy  LongPassing  BallControl  Acceleration  SprintSpeed  \
0   87.0        87.0         88.0         87.0          77.0         73.0   
1   76.0        75.0         84.0         87.0          78.0         83.0   
2   86.0        82.0         77.0         86.0          76.0         69.0   
3   85.0        83.0         93.0         91.0          76.0         76.0   
4   88.0        75.0         78.0         88.0          77.0         76.0   

   Agility  Reactions  Balance  ShotPower  Jumping  Stamina  Strength  \
0     80.0       91.0     79.0       89.0     73.0     91.0      70.0   
1     76.0       88.0     71.0       85.0     79.0     88.0      88.0   
2     75.0       92.0     78.0       89.0     69.0     78.0      85.0   
3     79.0       91.0     78.0       91.0     63.0     89.0      74.0   
4     83.0       83.0     90.0       82.0     63.0     90.0      80.0   

   LongShots  Ag

## Molta dimensionalitat

Per ara la millor manera de resoldreu es mitjançan la informació de la correlació. 

In [12]:
corr = df.corr()

In [13]:
corr["Best Position"].loc[(corr["Best Position"] > 0.1) | (corr["Best Position"] < -0.1)]

Special              -0.462000
Weak Foot            -0.158918
Skill Moves          -0.379409
Crossing             -0.349748
Finishing            -0.485977
HeadingAccuracy      -0.538797
ShortPassing         -0.541166
Volleys              -0.486789
Dribbling            -0.516102
Curve                -0.432627
FKAccuracy           -0.431036
LongPassing          -0.433935
BallControl          -0.566808
Acceleration         -0.223753
SprintSpeed          -0.214466
Agility              -0.264922
Reactions            -0.123770
Balance              -0.262832
ShotPower            -0.328345
Jumping              -0.105207
Stamina              -0.354369
Strength             -0.132240
LongShots            -0.511473
Aggression           -0.387180
Interceptions        -0.254471
Positioning          -0.508916
Vision               -0.348486
Penalties            -0.507632
Composure            -0.374373
Marking              -0.172155
StandingTackle       -0.277508
SlidingTackle        -0.244725
GKDiving